# Clustering Crypto

In [7]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from urllib.request import Request, urlopen
import json

### Fetching Cryptocurrency Data

In [9]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
json_request = Request(url)
json_response = urlopen(json_request)
data = json_response.read()
json_data = json.loads(data)

In [12]:
# Create a DataFrame 
crypto_df = pd.DataFrame(json_data['Data']).T
crypto_df.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,BlockTime,AssetLaunchDate,MaxSupply,MktCapPenalty,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,IsUsedInDefi
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,0,2017-07-01,300,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame

### Data Preprocessing

In [50]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'

new_crypto_df = crypto_df.drop(columns=['Id','Url','ImageUrl','ContentCreatedOn','Name','Symbol','FullName','Description', 'AssetTokenStatus','BlockTime','AssetLaunchDate','MaxSupply', 'MktCapPenalty', 'PlatformType', 'BuiltOn', 'SmartContractAddress', 'DecimalPoints', 'Difficulty'])


In [51]:
# Keep only cryptocurrencies that are trading
new_crypto_df = new_crypto_df.loc[new_crypto_df['IsTrading'] == True]

In [52]:
# Keep only cryptocurrencies with a working algorithm
new_crypto_df = new_crypto_df.loc[new_crypto_df['Algorithm'] != 'N/A']

In [53]:
# Remove the "IsTrading" column
new_crypto_df = new_crypto_df.drop(columns='IsTrading')

In [54]:
# Remove rows with at least 1 null value
new_crypto_df.dropna()
new_crypto_df.head(10)

,CoinName,Algorithm,ProofType,SortOrder,Sponsored,Taxonomy,Rating,TotalCoinsMined,BlockNumber,NetHashesPerSecond,BlockReward,IsUsedInDefi
42,42 Coin,Scrypt,PoW/PoS,34,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0,0,0,0,NaN
365,365Coin,X11,PoW/PoS,916,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0,0,0,0,NaN
404,404Coin,Scrypt,PoW/PoS,602,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0,0,0,0,NaN
611,SixEleven,SHA-256,PoW,586,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0,0,0,0,NaN
808,808,SHA-256,PoW/PoS,750,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0,0,0,0,NaN
1337,EliteCoin,X13,PoW/PoS,577,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0,0,0,0,NaN
2015,2015 coin,X11,PoW/PoS,33,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0,0,0,0,NaN
BTCD,BitcoinDark,SHA-256,PoW/PoS,13,False,"{'Access': 'Permissionless', 'FCA': 'Exchange'...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",NaN,NaN,NaN,NaN,NaN
CRAIG,CraigsCoin,X11,PoS,15,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",NaN,NaN,NaN,NaN,NaN
XBS,Bitstake,X11,PoW/PoS,16,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",NaN,NaN,NaN,NaN,NaN


In [55]:
# Remove rows with cryptocurrencies having no coins mined
new_crypto_df = new_crypto_df.loc[new_crypto_df['TotalCoinsMined'] != 0]

In [56]:
# Drop rows where there are 'N/A' text values
newest_crypto_df = new_crypto_df.notnull()

In [57]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df

coin_name_df = pd.DataFrame(newest_crypto_df["CoinName"])
coin_name_df.head()




,CoinName
BTCD,True
CRAIG,True
XBS,True
XPY,True
PRC,True


In [60]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
updated_df = newest_crypto_df.drop(["CoinName"], axis=1)

In [67]:
# Create dummy variables for text features
dummy_vars = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
dummy_vars.head(10)

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,ProofType_Scrypt-adaptive-N (ASIC resistant),ProofType_TPoS,ProofType_Tangle,ProofType_Zero-Knowledge Proof,ProofType_dPoS,ProofType_dPoS/BFT,ProofType_dPoW/PoW,ProofType_ePoW,ProofType_fPoW+ PBFT,ProofType_mFBA
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0,0,0,0,0,0,0,0,0,0
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,0,0,0,0,0,0,0,0,0,0
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0,0,0,0,0,0,0,0,0,0
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0,0,0,0,0,0,0,0,0,0
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,0,0,0,0,0,0,0,0,0,0
611,20909,/coins/611/overview,/media/35650940/611-sixeleven.png,1465914773,611,611,SixEleven,SixEleven (611),"611 is a Namecoin based cryptocurrency, and it...",N/A,...,0,0,0,0,0,0,0,0,0,0
808,28223,/coins/808/overview,/media/351513/808.png,1473980395,808,808,808,808 (808),808 is a coin develop for the music community ...,N/A,...,0,0,0,0,0,0,0,0,0,0
888,29462,/coins/888/overview,/media/351639/888.png,1475534352,888,888,Octocoin,Octocoin (888),Octocoin is a Proof of Work cryptocurrency. It...,N/A,...,0,0,0,0,0,0,0,0,0,0
1337,20824,/coins/1337/overview,/media/35520987/elite.png,1465838687,1337,1337,EliteCoin,EliteCoin (1337),1337 coin was created as an experimental Proof...,N/A,...,0,0,0,0,0,0,0,0,0,0
2015,3744,/coins/2015/overview,/media/20180/2015.png,1425316878,2015,2015,2015 coin,2015 coin (2015),Twenty15 Coin was conceived to be an asset bac...,N/A,...,0,0,0,0,0,0,0,0,0,0


In [73]:
# Standardize data
X_scaled = StandardScaler().fit_transform(dummy_vars)
print(X_scaled[0:5])

ValueError: could not convert string to float: '/coins/42/overview'

### Reducing Dimensions Using PCA

In [72]:
# Use PCA to reduce dimensions to 3 principal components


In [ ]:
# Create a DataFrame with the principal components data


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values


# Create the Elbow Curve using hvPlot


Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model

# Fit the model

# Predict clusters

# Create a new DataFrame including predicted clusters and cryptocurrencies features


### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters


#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
